<a href="https://colab.research.google.com/github/wjdgoruds2/Machine_Learning/blob/main/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost#catboost설치
import torch #pytorch
import pandas as pd#데이터 분석 위함
import numpy as np#벡터와 행렬 연산
from catboost import Pool,CatBoostClassifier#pool:catboost에서 모델 학습할 데이터 구조,CatBoostClassifier:catboost에서 분류 위함
from catboost.datasets import rotten_tomatoes#rotten_tomatoes데이터 사용

learn,_=rotten_tomatoes()#마지막 무시하고learn에서 rotten_tomatoes데이터 저장
print('Feature names=\n'+', '.join(list(learn)))#learn배열의 모든 요소 연결해 하나의 문자열로
auxiliary_colums=['id','theater_date','dvd_date','rating','date']#컬럼 설정
cat_features=['rating_MPAA','studio','fresh','critic','top_critic','publisher']#cat_features배열 설정
text_features=['synopsis','genre','director','writer','review']#text_features배열 설정

def get_processed_rotten_tomatoes():
  learn,test=rotten_tomatoes()#learn,test에 rotten_tomatoes데이터 저장

  def fill_na(df,features)
    for feature in features:#features만큼 반복
      df[feature].fillna('',inplace=True)#열의 빈값은 공백으로 채워 저장

  def preprocess_data_part(data_part):
    data_part=data_part.drop(auxiliary_colums,axis=1)#auxiliary_colums을 y축 기준으로 합 삭제

    fill_na(data_part,cat_features)#data_part에 cat_features칼럼
    fill_na(data_part,text_features)#data_part에 text_features칼럼

    X=data_part.drop(['rating_10'],axis=1)#X=rating_10을 y축 기준으로 합 삭제한 값
    y_train=data_part['rating_10']#y_train=rating_10값
    return X,y_train
  X_learn,y_learn=preprocess_data_part(learn)#learn데이터 결정
  X_test,y_test=preprocess_data_part(test)#test데이터 결정

  return X_learn,X_test,y_learn,y_test#X_learn,X_test,y_learn,y_test반환

X_train,X_test,y_train,y_test=get_processed_rotten_tomatoes()#X_train,X_test,y_train,y_test설정

def fit_catboost_on_rotten_tomatoes(X_train,X_test,y_train,y_test,catboost_params={},verbose=100):#100번 반복할때마다 알려줌
  learn_pool=Pool(#learn_pool기본값 설정
      X_train,
      y_train,
      cat_features=cat_features,
      text_features=text_features,
      feature_names=list(X_train)
  )
  test_pool=Pool(#test_pool기본값 설정
      X_test,
      y_test,
      cat_features=cat_features,
      text_features=text_features,
      feature_names=list(X_train)
  )
  catboost_default_params={#학습회수,경사하강법을 위한 learning_rate값,성능 평가 방법,#장치유형 설정
      'iterations':1000,
      'learning_rate':0.03,
      'eval_metric':'Accuracy',
      'task_type':'GPU',
 
  }
  catboost_default_params.update(catboost_params)#catboost_default_params을catboost_params로 설정

  model=CatBoostClassifier(**catboost_default_params)#catboost분류계산
  model.fit(learn_pool,eval_set=test_pool,verbose=verbose)#test_pool을 검증세트로 지정,learn_pool을 이 검증세트로 평가

  return model#model값 반환
fit_catboost_on_rotten_tomatoes(X_train,X_test,y_train,y_test)#X_train,X_test,y_train,y_test로 예측 계산 값 출력


     |████████████████████████████████| 67.3MB 46kB/s 
Feature names=
id, synopsis, rating_MPAA, genre, director, writer, theater_date, dvd_date, box_office, runtime, studio, dvd_date_int, theater_date_int, review, rating, fresh, critic, top_critic, publisher, date, date_int, rating_10
0:	learn: 0.3858217	test: 0.3980927	best: 0.3980927 (0)	total: 106ms	remaining: 1m 46s
100:	learn: 0.4458303	test: 0.4559237	best: 0.4561682 (99)	total: 5.21s	remaining: 46.4s
200:	learn: 0.4561323	test: 0.4605698	best: 0.4610588 (198)	total: 9.44s	remaining: 37.5s
300:	learn: 0.4639276	test: 0.4655826	best: 0.4657049 (298)	total: 13.2s	remaining: 30.7s
400:	learn: 0.4710504	test: 0.4675388	best: 0.4679056 (388)	total: 17.1s	remaining: 25.6s
500:	learn: 0.4785094	test: 0.4664384	best: 0.4686392 (408)	total: 21.1s	remaining: 21.1s
600:	learn: 0.4843788	test: 0.4681501	best: 0.4686392 (408)	total: 25s	remaining: 16.6s
700:	learn: 0.4900342	test: 0.4677833	best: 0.4690060 (666)	total: 28.9s	remaining: 12.3s